In [ ]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 140 kB 66.1 MB/s 
     |████████████████████████████████| 236 kB 50.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import spotipy.util as util
import pandas as pd
import time
import sys
import configparser
import os
import base64
import requests
import datetime
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from tabulate import tabulate

In [ ]:
scope = "user-library-read playlist-modify-private"
OAuth = SpotifyOAuth(
        scope=scope,
        redirect_uri='https://www.google.com/',
        client_id = '',
        client_secret = '',
        username= '',
        open_browser =False)



In [ ]:
sp = spotipy.Spotify(auth_manager=OAuth)

In [ ]:
offset = 0
songs = []
items = []
ids = []
while True:
    content = sp.current_user_saved_tracks(limit=50, offset=offset)
    songs += content['items']
    if content['next'] is not None:
        offset += 100
    else:
        break

In [ ]:
    for i in songs:
        ids.append(i['track']['id'])

    index = 0
    audio_features = []
    while index < len(ids):
        audio_features += sp.audio_features(ids[index:index + 50])
        index += 50
    features_list = []
    for features in audio_features:
        features_list.append([features['energy'], features['liveness'],
                              features['tempo'], features['speechiness'],
                              features['acousticness'], features['instrumentalness'],
                              features['time_signature'], features['danceability'],
                              features['key'], features['duration_ms'],
                              features['loudness'], features['valence'],
                              features['mode'], features['type'],
                              features['uri']])

    df = pd.DataFrame(features_list, columns=['energy', 'liveness',
                                              'tempo', 'speechiness',
                                              'acousticness', 'instrumentalness',
                                              'time_signature', 'danceability',
                                              'key', 'duration_ms', 'loudness',
                                              'valence', 'mode', 'type', 'uri'])
    print(tabulate(df))#,headers='firstrow'))
    df.to_csv('audio_features.csv', index=False)
    print(df.columns)



----  -------  ------  -------  ------  --------  --------  -  ------  --  ------  -------  ------  -  --------------  ------------------------------------
   0  0.378    0.117   104.125  0.0384  0.529     0.427     4  0.597    1  187788  -12.258  0.0958  0  audio_features  spotify:track:7hWC4zYwslDtvnTCqhzW0S
   1  0.751    0.114   115.976  0.0341  0.00369   0.0493    4  0.664    4  197512   -8.362  0.514   0  audio_features  spotify:track:1ITxluglNSDzhZf7kpgFDp
   2  0.333    0.115   130.552  0.0315  0.882     0.239     4  0.599    0  362432  -13.255  0.173   0  audio_features  spotify:track:2rkWdRk71jf5WtWXcs8qzw
   3  0.575    0.11     92.496  0.0267  0.0187    0.000185  4  0.527    2  269875   -8.044  0.224   1  audio_features  spotify:track:3If1PJtB9UqMf5VwCWHp80
   4  0.294    0.0869  119.906  0.0338  0.508     0.0766    4  0.328    9  246000  -13.849  0.0332  0  audio_features  spotify:track:6KgCvOoTI1GpYeWyj7o3Do
   5  0.802    0.0925  152.734  0.0418  0.076     0.00908   4  0

In [ ]:
df

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration_ms,loudness,valence,mode,type,uri
0,0.378,0.1170,104.125,0.0384,0.52900,0.427000,4,0.597,1,187788,-12.258,0.0958,0,audio_features,spotify:track:7hWC4zYwslDtvnTCqhzW0S
1,0.751,0.1140,115.976,0.0341,0.00369,0.049300,4,0.664,4,197512,-8.362,0.5140,0,audio_features,spotify:track:1ITxluglNSDzhZf7kpgFDp
2,0.333,0.1150,130.552,0.0315,0.88200,0.239000,4,0.599,0,362432,-13.255,0.1730,0,audio_features,spotify:track:2rkWdRk71jf5WtWXcs8qzw
3,0.575,0.1100,92.496,0.0267,0.01870,0.000185,4,0.527,2,269875,-8.044,0.2240,1,audio_features,spotify:track:3If1PJtB9UqMf5VwCWHp80
4,0.294,0.0869,119.906,0.0338,0.50800,0.076600,4,0.328,9,246000,-13.849,0.0332,0,audio_features,spotify:track:6KgCvOoTI1GpYeWyj7o3Do
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,0.566,0.0802,130.776,0.0574,0.16500,0.029100,4,0.796,0,188528,-8.484,0.7150,1,audio_features,spotify:track:2AgqXfsn2W4lXjchlqRyz1
3196,0.545,0.1190,113.962,0.0324,0.14000,0.000000,4,0.682,11,216160,-7.981,0.5920,1,audio_features,spotify:track:1kvGyV4fgzwCXxYfZ2ORiy
3197,0.842,0.1220,162.011,0.0843,0.00579,0.000000,4,0.494,7,229093,-4.784,0.5740,1,audio_features,spotify:track:2TfSHkHiFO4gRztVIkggkE
3198,0.712,0.1100,114.997,0.0323,0.08020,0.000000,4,0.722,6,359098,-6.099,0.5910,0,audio_features,spotify:track:3MO8BjfBXKGn3e3K9D0jix


In [ ]:
df = pd.read_csv("audio_features.csv")
df["genre"] = ""

for index, row in df.iterrows():
        uri = row['uri']
        track = sp.track(uri)
        artist_uri = track['album']['artists'][0]['uri']
        artist = sp.artist(artist_uri)

        genres = artist['genres']
        str_genres = ','.join([str(elem) for elem in genres])
        row['genre'] = str_genres
        df.loc[index, 'genre'] = row['genre']
        #print(df.loc[index,'genre'])
print(tabulate(df[50:100], headers='keys', tablefmt='psql'))
df.to_csv('final_audio_features.csv', index=False)

playlist = sp.current_user_playlists()
print(playlist['items'][1]['id'])